In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchvision import datasets, transforms
from torch.utils.data import random_split
from matplotlib import pyplot as plt

torch.manual_seed(123)

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Based on code from the pytorch tutorials

data_path = '../data/'
bp_filter = lambda x: x[1] in [0,2] # Bird and Plane Filter. Or if you will, Everything but Bird and Plane Filter
preprocessor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ])

cifar10_train_val = list(filter(bp_filter, datasets.CIFAR10(data_path, train=True, download=True, transform=preprocessor)))
cifar10_test = list(filter(bp_filter, datasets.CIFAR10(data_path, train=False, download=True, transform=preprocessor)))

n_train = int(len(cifar10_train_val)*0.90)
n_val =  len(cifar10_train_val) - n_train
cifar10_train, cifar10_val = random_split(
        cifar10_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print("Size of the train dataset:        ", len(cifar10_train))
print("Size of the validation dataset:   ", len(cifar10_val))
print("Size of the test dataset:         ", len(cifar10_test))

Size of the train dataset:         9000
Size of the validation dataset:    1000
Size of the test dataset:          2000


In [5]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()  # to inherit the '__init__' method from the 'nn.Module' class
        # Add whatever you want here (e.g layers and activation functions)
        # The order and names don't matter here but it is easier to understand
        # if you go for Layer1, fun1, layer2, fun2, etc
        # Some conventions:
        # - conv stands for convolution
        # - pool for pooling
        # - fc for fully connected

        self.flat = nn.Flatten()
        # 32*32*3: determined by our dataset: 32x32 RGB images
        self.fc1 = nn.Linear(32*32*3, 512)
        self.fc2 = nn.Linear(512, 128)
        self.act2 = nn.ReLU()
        # 2: determined by our number of classes (birds and planes)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 3)
        
    # Remember, we saw earlier that `forward` defines the 
    # computation performed at every call (the forward pass) and that it
    # should be overridden by all subclasses.
    def forward(self, x):
        # Now the order matters! 
        out = self.flat(x)
        out = self.act2(self.fc1(out))
        out = self.act2(self.fc2(out))
        out = self.act2(self.fc3(out))
        out = self.fc4(out)
        return out
        

In [6]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    
    n_batch = len(train_loader)
    
    # We'll store there the training loss for each epoch
    losses_train = []
    
    # Set the network in training mode
    model.train()
    
    # Re-initialize gradients, just in case the model has been inappropriately 
    # manipulated before the training
    optimizer.zero_grad(set_to_none=True)
    
    for epoch in range(1, n_epochs + 1): 
        
        # Training loss for the current epoch
        loss_train = 0

        # Loop over our dataset (in batches the data loader creates for us)
        for imgs, labels in train_loader:
            
            # Feed a batch into our model
            outputs = model(imgs)
            
            # Compute the loss we wish to minimize 
            # Note that by default, it is the mean loss that is computed
            # (so entire_batch_loss / batch_size)
            loss = loss_fn(outputs, labels) 
            
            
            # Perform the backward step. That is, compute the gradients of all parameters we want the network to learn
            loss.backward()
            
            # Update the model
            optimizer.step() 
            
            # Zero out gradients before the next round (or the end of training)
            optimizer.zero_grad() 

            # Update loss for this epoch
            # It is important to transform the loss to a number with .item()
            loss_train += loss.item()
            
        # Store current epoch loss. 
        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
            
    return losses_train

In [34]:
def train_manual_update(n_epochs, model, loss_fn, train_loader, lr=0.01, weight_decay=0.001, momentum_coeff=0.9):
    n_batch = len(train_loader)
    model.train()
    loss_train = []
    for epoch in range(1, n_epochs + 1):
        loss = 0
        for img, label in train_loader:
            output = model(img)
            loss = loss_fn(output, label)
            loss.backward()
            if epoch==1:
                vel = []
            with torch.no_grad():
                for i, p in enumerate(model.parameters()):
                    if epoch == 1:
                        vel.append(torch.zeros(p.data.size(), dtype=torch.float).to(device="cpu"))
                    if momentum_coeff > 0:
                        vel[i] = momentum_coeff * vel[i] + p.grad.data
                        tempy = lr*(weight_decay*p.data)
                        p.data = p.data - lr*vel[i] - tempy
                    else:
                        tempy = lr*(weight_decay*p.data)
                        p.data = p.data - (lr*p.grad.data) - tempy
                    p.grad.data.zero_()
                    p.grad.detach_()
        loss_train.append(loss/n_batch)
        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss / n_batch))
    return loss_train

        

In [8]:
model = MyMLP()

optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

loss_train = train(3, optimizer, model, loss_fn, train_loader)
loss_train



13:34:41.273918  |  Epoch 1  |  Training loss 0.840


[0.8402051333840012, 0.5614196611634383, 0.4910872927371492]

In [36]:
model_manual = MyMLP()

optimizer = optim.SGD(model_manual.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

loss_train = train_manual_update(3, model_manual, loss_fn, train_loader)
loss_train


TypeError: unsupported operand type(s) for /: 'list' and 'int'